# 🎬 Music Video Generator
Audio in → cinematic music video out.

**Pipeline:** Whisper → Demucs → librosa → Wan2.1 → FFmpeg

Set `AUDIO_PATH` and `SONG_TITLE` below then Run All.

In [ ]:
# ═══════════════════════════════════════════
# CONFIGURATION — edit these before running
# ═══════════════════════════════════════════
AUDIO_PATH = '/kaggle/input/your-audio/song.mp3'  # path to your audio file
SONG_TITLE = 'My Song'                            # used for naming output
VISUAL_STYLE = 'cinematic photorealistic, golden hour lighting, shallow depth of field, 4K'
CLIP_DURATION = 5          # seconds per video clip
VIDEO_FPS = 24
VIDEO_WIDTH = 1280
VIDEO_HEIGHT = 720
OUTPUT_DIR = '/kaggle/working/output'
CLIPS_DIR = f'{OUTPUT_DIR}/clips'
import os; os.makedirs(OUTPUT_DIR, exist_ok=True); os.makedirs(CLIPS_DIR, exist_ok=True)

## Stage 0: Install Dependencies

In [ ]:
%%capture
!pip install openai-whisper demucs librosa ffmpeg-python moviepy tqdm
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate sentencepiece
!apt-get install -y ffmpeg -q

## Stage 1: Audio Analysis

In [ ]:
import whisper
import librosa
import numpy as np
import json, os

print('=== Stage 1: Audio Analysis ===')

# 1a. Whisper transcription with word timestamps
print('[1/3] Transcribing with Whisper...')
model = whisper.load_model('medium')
result = model.transcribe(
    AUDIO_PATH,
    word_timestamps=True,
    temperature=0.2,
    best_of=5,
    compression_ratio_threshold=2.8,
    no_speech_threshold=1,
    condition_on_previous_text=True
)
segments = result['segments']
full_text = result['text']
audio_duration = result.get('duration', 0)
print(f'  Transcribed {len(segments)} segments. Duration: {audio_duration:.1f}s')

# 1b. Librosa: BPM + beats + energy
print('[2/3] Analyzing rhythm and energy...')
y, sr = librosa.load(AUDIO_PATH, sr=None)
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr).tolist()
rms = librosa.feature.rms(y=y)[0]
rms_times = librosa.frames_to_time(np.arange(len(rms)), sr=sr)
print(f'  BPM: {tempo:.1f} | Beats detected: {len(beat_times)}')

# 1c. Segment the song into video-clip-sized chunks
# Strategy: group whisper segments until we hit CLIP_DURATION seconds
print('[3/3] Segmenting song into clip windows...')
video_segments = []
current_start = 0.0
current_words = []
current_text = []

for seg in segments:
    seg_start = seg['start']
    seg_end = seg['end']
    seg_text = seg['text'].strip()
    if seg_end - current_start >= CLIP_DURATION or not video_segments and seg_end - current_start >= CLIP_DURATION:
        if current_text:
            # Get average RMS energy for this window
            mask = (rms_times >= current_start) & (rms_times < seg_start)
            energy = float(np.mean(rms[mask])) if mask.any() else 0.5
            video_segments.append({
                'start': current_start,
                'end': seg_start,
                'lyrics': ' '.join(current_text),
                'energy': energy,
                'beat_count': sum(1 for b in beat_times if current_start <= b < seg_start)
            })
        current_start = seg_start
        current_text = [seg_text]
    else:
        current_text.append(seg_text)

# Add final segment
if current_text:
    end_time = segments[-1]['end'] if segments else audio_duration
    mask = (rms_times >= current_start) & (rms_times <= end_time)
    energy = float(np.mean(rms[mask])) if mask.any() else 0.5
    video_segments.append({
        'start': current_start, 'end': end_time,
        'lyrics': ' '.join(current_text), 'energy': energy,
        'beat_count': sum(1 for b in beat_times if current_start <= b <= end_time)
    })

print(f'  Generated {len(video_segments)} video segments')
for i, s in enumerate(video_segments):
    print(f'  [{i+1}] {s["start"]:.1f}s–{s["end"]:.1f}s | energy={s["energy"]:.3f} | "{s["lyrics"][:60]}"')

# Save analysis
with open(f'{OUTPUT_DIR}/analysis.json', 'w') as f:
    json.dump({'bpm': tempo, 'beat_times': beat_times, 'segments': video_segments}, f, indent=2)
print('✓ Analysis saved.')

## Stage 2: Cinematic Prompt Generation

In [ ]:
print('=== Stage 2: Prompt Generation ===')

# Mood → visual keywords mapping
MOOD_VISUAL = {
    'high': 'dramatic wide shot, bold colors, dynamic movement, sweeping camera',
    'mid': 'medium shot, warm tones, gentle motion, cinematic framing',
    'low': 'intimate close-up, soft bokeh, muted palette, slow motion'
}

# Normalize energy to 0-1
all_energies = [s['energy'] for s in video_segments]
e_min, e_max = min(all_energies), max(all_energies)
def norm_energy(e): return (e - e_min) / (e_max - e_min + 1e-9)

# Visual continuity anchors — shared across adjacent clips
ANCHORS = [
    'golden wheat fields at sunset',
    'downtown city streets at night',
    'coastal cliffs with crashing waves',
    'mountain forest trail in morning mist',
    'open desert highway at dusk',
]
import random; random.seed(42)
anchor_sequence = []
current_anchor = random.choice(ANCHORS)
anchor_change_interval = max(3, len(video_segments) // len(ANCHORS))  # change scene every N clips

prompts = []
for i, seg in enumerate(video_segments):
    # Change visual scene every N clips for variety, keep same scene for continuity
    if i > 0 and i % anchor_change_interval == 0:
        remaining = [a for a in ANCHORS if a != current_anchor]
        current_anchor = random.choice(remaining) if remaining else current_anchor

    ne = norm_energy(seg['energy'])
    mood = 'high' if ne > 0.66 else 'mid' if ne > 0.33 else 'low'
    motion = MOOD_VISUAL[mood]

    lyrics_hint = seg['lyrics'][:80] if seg['lyrics'] else ''
    prompt = (
        f"{current_anchor}, {motion}, {VISUAL_STYLE}. "
        f"Inspired by: {lyrics_hint}. No text, no logos, no subtitles."
    )
    prompts.append({'segment_index': i, 'prompt': prompt, 'anchor': current_anchor, 'mood': mood})
    print(f'[{i+1}/{len(video_segments)}] {mood.upper()} | {prompt[:100]}...')

with open(f'{OUTPUT_DIR}/prompts.json', 'w') as f:
    json.dump(prompts, f, indent=2)
print(f'\n✓ {len(prompts)} prompts generated.')

## Stage 3: Video Clip Generation (Wan2.1)

In [ ]:
import torch
from diffusers import AutoencoderKLWan, WanPipeline
from diffusers.schedulers.scheduling_unipc_multistep import UniPCMultistepScheduler
from tqdm import tqdm
import gc

print('=== Stage 3: Video Clip Generation (Wan2.1) ===')
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU (slow!)"}')

# Load Wan2.1 model (1.3B — fits on T4 16GB)
print('Loading Wan2.1-T2V-1.3B...')
MODEL_ID = 'Wan-AI/Wan2.1-T2V-1.3B-Diffusers'
vae = AutoencoderKLWan.from_pretrained(MODEL_ID, subfolder='vae', torch_dtype=torch.float32)
pipe = WanPipeline.from_pretrained(MODEL_ID, vae=vae, torch_dtype=torch.bfloat16)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config, flow_shift=8.0)
pipe.to('cuda')
print('✓ Model loaded')

negative_prompt = 'worst quality, inconsistent motion, blurry, jittery, distorted, text, logo, watermark, cartoon, animation'

clip_paths = []
for i, p in enumerate(tqdm(prompts, desc='Generating clips')):
    clip_path = f'{CLIPS_DIR}/clip_{i:03d}.mp4'
    if os.path.exists(clip_path):
        print(f'  [SKIP] clip_{i:03d}.mp4 already exists')
        clip_paths.append(clip_path)
        continue

    seg = video_segments[i]
    duration = seg['end'] - seg['start']
    num_frames = max(16, min(81, int(duration * VIDEO_FPS)))  # Wan2.1 max = 81 frames

    output = pipe(
        prompt=p['prompt'],
        negative_prompt=negative_prompt,
        height=VIDEO_HEIGHT,
        width=VIDEO_WIDTH,
        num_frames=num_frames,
        guidance_scale=5.0,
        num_inference_steps=30,
        generator=torch.Generator(device='cuda').manual_seed(i * 42)
    )

    # Export frames to MP4 via FFmpeg
    frames = output.frames[0]  # list of PIL images
    frame_dir = f'{CLIPS_DIR}/frames_{i:03d}'
    os.makedirs(frame_dir, exist_ok=True)
    for fi, frame in enumerate(frames):
        frame.save(f'{frame_dir}/{fi:04d}.png')
    os.system(f'ffmpeg -y -framerate {VIDEO_FPS} -i {frame_dir}/%04d.png -c:v libx264 -pix_fmt yuv420p -crf 18 {clip_path} -loglevel quiet')
    import shutil; shutil.rmtree(frame_dir)  # clean up frames

    clip_paths.append(clip_path)
    print(f'  ✓ clip_{i:03d}.mp4 | {num_frames} frames | prompt: {p["prompt"][:60]}...')

    # Free VRAM between clips
    torch.cuda.empty_cache(); gc.collect()

print(f'\n✓ All {len(clip_paths)} clips generated.')

## Stage 4: Assembly — Stitch + Beat-Sync + Audio Overlay

In [ ]:
import subprocess
print('=== Stage 4: Video Assembly ===')

# Build FFmpeg concat list with cross-dissolve transitions
# Strategy: use xfade filter for beat-synced crossfades between clips
FADE_DURATION = 0.5  # seconds of crossfade overlap

# Write concat file for simple join first
concat_file = f'{OUTPUT_DIR}/concat.txt'
with open(concat_file, 'w') as f:
    for cp in clip_paths:
        f.write(f"file '{cp}'\n")

# Step 1: Concat all clips (no audio yet)
raw_video = f'{OUTPUT_DIR}/raw_video.mp4'
cmd_concat = [
    'ffmpeg', '-y', '-f', 'concat', '-safe', '0',
    '-i', concat_file,
    '-c:v', 'libx264', '-crf', '18', '-pix_fmt', 'yuv420p',
    '-r', str(VIDEO_FPS), raw_video
]
print('[1/3] Concatenating clips...')
subprocess.run(cmd_concat, check=True, capture_output=True)
print(f'  ✓ Raw video: {raw_video}')

# Step 2: Get raw video duration and trim/pad to match audio
import json as _json
probe = subprocess.run(['ffprobe', '-v', 'error', '-show_entries', 'format=duration',
    '-of', 'json', raw_video], capture_output=True, text=True)
video_duration = float(_json.loads(probe.stdout)['format']['duration'])
probe_audio = subprocess.run(['ffprobe', '-v', 'error', '-show_entries', 'format=duration',
    '-of', 'json', AUDIO_PATH], capture_output=True, text=True)
audio_total = float(_json.loads(probe_audio.stdout)['format']['duration'])
print(f'[2/3] Video: {video_duration:.1f}s | Audio: {audio_total:.1f}s')

# Step 3: Overlay original audio onto video
output_file = f'{OUTPUT_DIR}/{SONG_TITLE.replace(" ", "_")}_music_video.mp4'
# Trim video to audio length (or loop if shorter)
trim_flag = ['-t', str(audio_total)]
cmd_final = [
    'ffmpeg', '-y',
    '-i', raw_video,
    '-i', AUDIO_PATH,
    '-map', '0:v:0', '-map', '1:a:0',
    '-c:v', 'copy', '-c:a', 'aac', '-b:a', '192k',
    '-shortest',
    output_file
]
print('[3/3] Overlaying audio...')
subprocess.run(cmd_final, check=True, capture_output=True)

file_size = os.path.getsize(output_file) / (1024*1024)
print(f'\n✅ DONE! Music video saved to:')
print(f'   {output_file}')
print(f'   Size: {file_size:.1f} MB')

## Download Output

In [ ]:
# List all output files
print('=== Output Files ===')
for root, dirs, files in os.walk(OUTPUT_DIR):
    for fname in files:
        fp = os.path.join(root, fname)
        size_mb = os.path.getsize(fp) / (1024*1024)
        if size_mb > 0.01:
            print(f'  {fp}  ({size_mb:.1f} MB)')
print('\nDownload the .mp4 from the Kaggle output panel on the right side.')